dataset link on kaggle : https://www.kaggle.com/datasets/stackoverflow/stacksample

In [70]:
import numpy as np 
import pandas as pd 
import os
from bs4 import BeautifulSoup
import re
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from tqdm import tqdm
import joblib


**Reading Data**

In [2]:
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/stacksample/Answers.csv
/kaggle/input/stacksample/Questions.csv
/kaggle/input/stacksample/Tags.csv


In [3]:
data_path = "/kaggle/input/stacksample/"

In [4]:
questions = pd.read_csv(data_path + "Questions.csv", encoding='latin1')
print(questions.shape)

(1264216, 7)


In [5]:
questions.head(10)

,Id,OwnerUserId,CreationDate,ClosedDate,Score,Title,Body
0,80,26.0,2008-08-01T13:57:07Z,NaN,26,SQLStatement.execute() - multiple queries in o...,<p>I've written a database generation script i...
1,90,58.0,2008-08-01T14:41:24Z,2012-12-26T03:45:49Z,144,Good branching and merging tutorials for Torto...,<p>Are there any really good tutorials explain...
2,120,83.0,2008-08-01T15:50:08Z,NaN,21,ASP.NET Site Maps,<p>Has anyone got experience creating <strong>...
3,180,2089740.0,2008-08-01T18:42:19Z,NaN,53,Function for creating color wheels,<p>This is something I've pseudo-solved many t...
4,260,91.0,2008-08-01T23:22:08Z,NaN,49,Adding scripting functionality to .NET applica...,<p>I have a little game written in C#. It uses...
5,330,63.0,2008-08-02T02:51:36Z,NaN,29,Should I use nested classes in this case?,<p>I am working on a collection of classes use...
6,470,71.0,2008-08-02T15:11:47Z,2016-03-26T05:23:29Z,13,Homegrown consumption of web services,<p>I've been writing a few web services for a ...
7,580,91.0,2008-08-02T23:30:59Z,NaN,21,Deploying SQL Server Databases from Test to Live,<p>I wonder how you guys manage deployment of ...
8,650,143.0,2008-08-03T11:12:52Z,NaN,79,Automatically update version number,<p>I would like the version property of my app...
9,810,233.0,2008-08-03T20:35:01Z,NaN,9,Visual Studio Setup Project - Per User Registr...,<p>I'm trying to maintain a Setup Project in <...


In [6]:
#answers = pd.read_csv(data_path + "Answers.csv", encoding='latin1')
#print(answers.shape)

In [7]:
#answers.head(10)

In [8]:
tags = pd.read_csv(data_path + "Tags.csv", encoding='latin1')
print(tags.shape)

(3750994, 2)


In [9]:
tags.head(10)

,Id,Tag
0,80,flex
1,80,actionscript-3
2,80,air
3,90,svn
4,90,tortoisesvn
5,90,branch
6,90,branching-and-merging
7,120,sql
8,120,asp.net
9,120,sitemap


In [10]:
questions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1264216 entries, 0 to 1264215
Data columns (total 7 columns):
 #   Column        Non-Null Count    Dtype  
---  ------        --------------    -----  
 0   Id            1264216 non-null  int64  
 1   OwnerUserId   1249762 non-null  float64
 2   CreationDate  1264216 non-null  object 
 3   ClosedDate    55959 non-null    object 
 4   Score         1264216 non-null  int64  
 5   Title         1264216 non-null  object 
 6   Body          1264216 non-null  object 
dtypes: float64(1), int64(2), object(4)
memory usage: 67.5+ MB


In [11]:
print(questions.notnull().sum()/questions.shape[0])

Id              1.000000
OwnerUserId     0.988567
CreationDate    1.000000
ClosedDate      0.044264
Score           1.000000
Title           1.000000
Body            1.000000
dtype: float64


In [12]:
#answers.info()

In [13]:
#print(answers.notnull().sum()/answers.shape[0])

In [14]:
tags.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3750994 entries, 0 to 3750993
Data columns (total 2 columns):
 #   Column  Dtype 
---  ------  ----- 
 0   Id      int64 
 1   Tag     object
dtypes: int64(1), object(1)
memory usage: 57.2+ MB


In [15]:
print(tags.notnull().sum()/tags.shape[0])

Id     1.000000
Tag    0.999703
dtype: float64


**preprocessing**

Nulls

In [16]:
tags=tags.dropna()

Number of unique tags

In [17]:
tqdm.pandas()

In [18]:
len(tags['Tag'].unique())

37034

In [19]:
sorted_tags = tags.groupby('Tag').size().reset_index(name='Count') \
                 .sort_values('Count', ascending=False)

In [20]:
print(sorted_tags.tail(30000))

                 Tag  Count
3409   bidirectional     34
28663     selectlist     34
3360     berkeley-db     34
19287            mdf     34
12652          gmock     34
...              ...    ...
31989           tbcd      1
10167     evil-dicom      1
10169            evo      1
31986           tbar      1
24456         pjsua2      1

[30000 rows x 2 columns]


In [21]:
top_tags = sorted_tags.head(100)['Tag']
filtered_tags = tags[tags['Tag'].isin(top_tags)]

In [22]:
print(filtered_tags.head())

     Id        Tag
7   120        sql
8   120    asp.net
10  180  algorithm
14  260         c#
15  260       .net


1057478

In [23]:
print(len(filtered_tags['Id'].unique()))

1057478


wrong approach

In [24]:
#string_tags_df = tags.groupby("Id", as_index=False)["Tag"].agg(" ".join)

In [25]:
#len(string_tags_df)

In [26]:
#string_tags_df.head(5)

In [27]:
#len(string_tags_df['Tag'].unique())

In [28]:
#full_tag_counts = string_tags_df['Tag'].value_counts().reset_index(name='Count')
#full_tag_counts.columns = ['Tag', 'Count']
#sorted_full_tag_counts = full_tag_counts.sort_values('Count', ascending=False)

In [29]:
#print(sorted_full_tag_counts.tail(650000))

In [30]:
#count_other_tags = sorted_full_tag_counts.iloc[1000:]['Count'].sum()
#count_other_tags

In [31]:
#answer_counts = answers.groupby("ParentId").size().to_dict()
#value_counts = Counter(answer_counts.values())
#sorted_counts = dict(sorted(value_counts.items()))
#print(sorted_counts)

**not include answers with zero or negative score**

In [32]:
#answer_counts = answers[answers['Score']>0].groupby("ParentId").size().to_dict()
#value_counts = Counter(answer_counts.values())
#sorted_counts = dict(sorted(value_counts.items()))
#print(sorted_counts)

add new column is the questions has link or now and the question after remove html tags

In [33]:
def remove_html_tags(html):
    soup = BeautifulSoup(html, 'html.parser')
    has_link = soup.find('a', href=True) is not None 
    text = soup.get_text(separator=' ', strip=True)
    return has_link, text

hyperparameters to play with

In [34]:
#questions = questions[questions['Score']]
len(questions)


1264216

In [35]:
#answer_counts = answers[answers['Score']>0]

***feature engineering***

In [36]:
questions['has_link'], questions['clean_text'] = zip(*questions['Body'].progress_apply(remove_html_tags))


100%|██████████| 1264216/1264216 [09:25<00:00, 2236.07it/s]


In [37]:
questions['combined_text'] = questions['Title'] + " " + questions['clean_text']

In [38]:
questions.drop(columns=['Body','Title'], inplace=True) # can make it questions if we will use it 

In [39]:
#answers['has_link'], answers['clean_text'] = zip(*answers['Body'].apply(remove_html_tags))

In [40]:
#answers.drop(columns=['Body'], inplace=True) 

In [41]:
questions['text_length'] = questions['clean_text'].apply(len)

In [42]:
questions['word_count'] = questions['clean_text'].apply(lambda x: len(x.split()))

In [43]:
programming_terms = {
        'javascript', 'java', 'c#', 'php', 'android', 'jquery',
        'python', 'html', 'c++', 'ios', 'mysql', 'css', 'sql',
        'asp.net', 'objective-c', 'ruby-on-rails', '.net', 'c',
        'iphone', 'angularjs', 'arrays', 'sql-server', 'json',
        'ruby', 'r', 'ajax', 'regex', 'xml', 'node.js',
        'asp.net-mvc', 'linux', 'django', 'wpf', 'database', 'swift'
    }

In [44]:
programming_terms = {term.lower() for term in programming_terms}


In [45]:
def clean_preserve_programming_terms(text):
    words = text.lower().split()
    cleaned = [
        word if word in programming_terms else re.sub(r'[^\w\s]', '', word)
        for word in words
    ]
    return ' '.join(cleaned)

In [46]:
questions['cleaned_text'] = questions['combined_text'].progress_apply(clean_preserve_programming_terms)


100%|██████████| 1264216/1264216 [03:35<00:00, 5855.08it/s]


we can add new feature by add column of numbers of answers 

In [47]:
#answers_grouped = answers.groupby('ParentId')['Body'].apply(lambda x: ' '.join(x)).reset_index()
#answers_grouped.columns = ['Id', 'AllAnswers']

In [48]:
#questions = questions.merge(answers_grouped, on='Id', how='left')

In [49]:
print(questions.notnull().sum()/questions.shape[0])

Id               1.000000
OwnerUserId      0.988567
CreationDate     1.000000
ClosedDate       0.044264
Score            1.000000
has_link         1.000000
clean_text       1.000000
combined_text    1.000000
text_length      1.000000
word_count       1.000000
cleaned_text     1.000000
dtype: float64


In [50]:
#print(answers.notnull().sum()/answers.shape[0])

In [51]:
grouped_tags = filtered_tags.groupby('Id')['Tag'].apply(list).reset_index(name='tags')


In [52]:
grouped_tags.head()

,Id,tags
0,120,"[sql, asp.net]"
1,180,[algorithm]
2,260,"[c#, .net]"
3,330,"[c++, class]"
4,470,"[.net, web-services]"


In [53]:
merged_df = pd.merge(questions, grouped_tags, on='Id', how='inner')


In [54]:
merged_df.head()

,Id,OwnerUserId,CreationDate,ClosedDate,Score,has_link,clean_text,combined_text,text_length,word_count,cleaned_text,tags
0,120,83.0,2008-08-01T15:50:08Z,NaN,21,False,Has anyone got experience creating SQL-based A...,ASP.NET Site Maps Has anyone got experience cr...,347,56,asp.net site maps has anyone got experience cr...,"[sql, asp.net]"
1,180,2089740.0,2008-08-01T18:42:19Z,NaN,53,False,This is something I've pseudo-solved many time...,Function for creating color wheels This is som...,226,41,function for creating color wheels this is som...,[algorithm]
2,260,91.0,2008-08-01T23:22:08Z,NaN,49,True,I have a little game written in C#. It uses a ...,Adding scripting functionality to .NET applica...,1037,178,adding scripting functionality to .net applica...,"[c#, .net]"
3,330,63.0,2008-08-02T02:51:36Z,NaN,29,False,I am working on a collection of classes used f...,Should I use nested classes in this case? I am...,948,170,should i use nested classes in this case i am ...,"[c++, class]"
4,470,71.0,2008-08-02T15:11:47Z,2016-03-26T05:23:29Z,13,False,I've been writing a few web services for a .ne...,Homegrown consumption of web services I've bee...,300,52,homegrown consumption of web services ive been...,"[.net, web-services]"


In [55]:
merged_df.columns

Index(['Id', 'OwnerUserId', 'CreationDate', 'ClosedDate', 'Score', 'has_link',
       'clean_text', 'combined_text', 'text_length', 'word_count',
       'cleaned_text', 'tags'],
      dtype='object')

In [56]:
merged_df.drop(['Id', 'OwnerUserId', 'CreationDate', 'ClosedDate',  'has_link','combined_text',
       'clean_text'],axis=True)

,Score,text_length,word_count,cleaned_text,tags
0,21,347,56,asp.net site maps has anyone got experience cr...,"[sql, asp.net]"
1,53,226,41,function for creating color wheels this is som...,[algorithm]
2,49,1037,178,adding scripting functionality to .net applica...,"[c#, .net]"
3,29,948,170,should i use nested classes in this case i am ...,"[c++, class]"
4,13,300,52,homegrown consumption of web services ive been...,"[.net, web-services]"
...,...,...,...,...,...
1057473,0,325,53,how to bind a model to resource store method i...,[laravel]
1057474,1,411,76,how to execute multiline python code from a ba...,"[python, bash]"
1057475,0,1094,144,url routing in php mvc i am building a custom ...,"[php, .htaccess]"
1057476,1,524,76,obfuscating code in android studio under minif...,[android]


In [57]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()


In [58]:
y = mlb.fit_transform(merged_df['tags'])

In [59]:
binary_df = pd.DataFrame(y, columns=mlb.classes_)

In [60]:
binary_df

,.htaccess,.net,ajax,algorithm,android,angularjs,apache,api,arrays,asp.net,...,visual-studio,visual-studio-2010,wcf,web-services,windows,winforms,wordpress,wpf,xcode,xml
0,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,1,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1057473,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1057474,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1057475,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1057476,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


***tf_idf***

In [61]:
tfidf = TfidfVectorizer(
    max_features=20000,# could change but takecare of crash
    stop_words='english',
    ngram_range=(1, 2),# you can make it 2 but could get crash due to ram
)

In [62]:
clean_texts = list(tqdm(merged_df['combined_text'], desc="Loading text data"))


Loading text data: 100%|██████████| 1057478/1057478 [00:00<00:00, 2901156.77it/s]


In [63]:
X = tfidf.fit_transform(tqdm(clean_texts, desc="TF-IDF Vectorizing"))

TF-IDF Vectorizing: 100%|██████████| 1057478/1057478 [04:30<00:00, 3916.38it/s]


donot forget label encoding

In [64]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, binary_df, test_size=0.2, random_state=42
)

models , classifier   

In [65]:
# from sklearn.preprocessing import StandardScaler

# scaler = StandardScaler(with_mean=False)
# X_train_scaled = scaler.fit_transform(X_train)
# X_test_scaled = scaler.transform(X_test)


In [66]:
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier

clf = OneVsRestClassifier(LogisticRegression(solver='saga', max_iter=1500))
clf.fit(X_train, y_train)


OneVsRestClassifier(estimator=LogisticRegression(max_iter=1500, solver='saga'))

In [67]:
y_pred = clf.predict(X_test)
y_train_pred = clf.predict(X_train)
from sklearn.metrics import accuracy_score, f1_score

accuracy = accuracy_score(y_test, y_pred)
train_accuracy = accuracy_score(y_train, y_train_pred)
print(f"Train Accuracy: {train_accuracy*100}")
print(f"Test Accuracy: {accuracy*100}")

Train Accuracy: 39.29149792785189
Test Accuracy: 37.220562091008816


In [71]:
joblib.dump(clf, 'logistic_ovr_model.pkl')

['logistic_ovr_model.pkl']